### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
from pandas import json_normalize


In [2]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [3]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = f"{url}fq={filter_query}&sort={sort}&fl={field_list}&begin_date={begin_date}&end_date={end_date}&api-key={nyt_api_key}"


In [4]:
response = requests.get(query_url)
response

<Response [200]>

In [6]:

# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(0,20):
    # create query with a page number
    # API results show 10 articles at a time
    
    loop_url = f"{query_url}&page={page}"
    
    
    # Make a "GET" request and retrieve the JSON
    reviews = requests.get(loop_url).json()
    

    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list

    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in reviews['response']['docs']:
            reviews_list.append(review)
        

        # Print the page that was just retrieved
        print(f'Page{page} was retrieved successfully')

        # Print the page number that had no results then break from the loop
    except:
        print(f'No results found on {page}')
    
    
print(f"Total reviews retrieved: {len(reviews_list)}")



Page0 was retrieved successfully
Page1 was retrieved successfully
Page2 was retrieved successfully
Page3 was retrieved successfully
Page4 was retrieved successfully
Page5 was retrieved successfully
Page6 was retrieved successfully
Page7 was retrieved successfully
Page8 was retrieved successfully
Page9 was retrieved successfully
Page10 was retrieved successfully
Page11 was retrieved successfully
Page12 was retrieved successfully
Page13 was retrieved successfully
Page14 was retrieved successfully
Page15 was retrieved successfully
Page16 was retrieved successfully
Page17 was retrieved successfully
Page18 was retrieved successfully
Page19 was retrieved successfully
Total reviews retrieved: 200


In [7]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[0:5], indent=4))


[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [8]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews_list)
reviews_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None


In [9]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
extract_title = lambda st: st[st.find("\u2018")+1:st.rfind("\u2019")]
reviews_df['title'] = reviews_df['headline.main'].apply(extract_title)




In [10]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    if isinstance(keyword_list, list):
        for item in keyword_list:
        # Extract 'name' and 'value'
            keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
            extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df['keywords'] = reviews_df['keywords'].apply(extract_keywords)


In [11]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles_list = reviews_df['title'].to_list()
titles_list

['The Attachment Diaries',
 'What’s Love Got to Do With It?',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate’ Review: Love the One You',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder’ Review: A God',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction’ Review: A Poet',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 

### Access The Movie Database API

In [12]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [13]:
# Create an empty list to store the results
tmdb_movies_list = [] 

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 0 

# Loop through the titles
for title in titles_list:
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0 and request_counter != 0:
        print("Reached a multiple of 50 requests. Sleeping for a while...")
        time.sleep(5)

    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    search_url = f"https://api.themoviedb.org/3/search/movie?query={title}&api_key={tmdb_api_key}"
    response = requests.get(search_url)
    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Get movie id
        movie_id = response.json()["results"][0]["id"]
        movie_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
        movie_response = requests.get(movie_url)
        movie_data = movie_response.json()
        # Make a request for a the full movie details
        #movie_response = requests.get(movie_url)

        # Execute "GET" request with url
        movie_response = requests.get(movie_url)
        
        # Extract the genre names into a list
        genres = [genre['name'] for genre in movie_data.get('genres', [])]

        # Extract the spoken_languages' English name into a list
        spoken_languages = [lang["english_name"] for lang in movie_data["spoken_languages"]]

        # Extract the production_countries' name into a list
        production_countries = [country['name'] for country in movie_data.get('production_countries', [])]

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        movie_info = {
            "title": movie_data["title"],
            "original_title": movie_data["original_title"],
            "budget": movie_data["budget"],
            "original_language": movie_data["original_language"],
            "homepage": movie_data["homepage"],
            "overview": movie_data["overview"],
            "popularity": movie_data["popularity"],
            "runtime": movie_data["runtime"],
            "revenue": movie_data["revenue"],
            "release_date": movie_data["release_date"],
            "vote_average": movie_data["vote_average"],
            "vote_count": movie_data["vote_count"],
            "genres": genres,
            "spoken_languages": spoken_languages,
            "production_countries": production_countries
        }
        tmdb_movies_list.append(movie_info)
        # Print out the title that was found
        print(f"Title found: {movie_data['title']}")
    except IndexError:
        print(f"Movie not found for title: {title}")
        
    



Title found: The Attachment Diaries
Title found: What's Love Got to Do with It?
Title found: You Can Live Forever
Title found: A Tourist's Guide to Love
Title found: Other People's Children
Title found: One True Loves
Title found: The Lost Weekend: A Love Story
Title found: A Thousand and One
Title found: Your Place or Mine
Title found: Love in the Time of Fentanyl
Title found: Pamela, A Love Story
Title found: In from the Side
Title found: After Love
Title found: Alcarràs
Title found: Nelly and Monsieur Arnaud
Title found: Lady Chatterley's Lover
Title found: The Sound of Christmas
Title found: The Inspection
Title found: Bones and All
Title found: My Policeman
Movie not found for title: About Fate’ Review: Love the One You
Title found: Waiting for Bojangles
Title found: I Love My Dad
Title found: A Love Song
Title found: Alone Together
Title found: Art of Love
Title found: 2099: The Soldier Protocol
Movie not found for title: Thor: Love and Thunder’ Review: A God
Title found: Both Si

In [24]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[0:5], indent=4))


[
    {
        "title": "The Attachment Diaries",
        "original_title": "El apego",
        "budget": 0,
        "original_language": "es",
        "homepage": "",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 2.89,
        "runtime": 102,
        "revenue": 0,
        "release_date": "2021-10-07",
        "vote_average": 3.0,
        "vote_count": 4,
        "genres": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "spoken_languages": [
            "Spanish"
        ],
        "production_countries": [
            "Argentina"
        ]
  

In [25]:
# Convert the results to a DataFrame
movies_df = pd.json_normalize(tmdb_movies_list)
movies_df

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",2.890,102,0,2021-10-07,3.000,4,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,What's Love Got to Do with It?,What's Love Got to Do with It?,0,en,,Two childhood friends now in their thirties mu...,13.218,109,10898395,2023-01-26,6.075,174,"[Romance, Comedy]","[English, Portuguese, Urdu]","[France, United Kingdom]"
2,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",36.659,96,15055,2023-03-24,6.622,37,"[Drama, Romance]","[English, French]","[Canada, United States of America]"
3,A Tourist's Guide to Love,A Tourist's Guide to Love,0,en,https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",12.581,96,0,2023-04-21,6.289,161,"[Romance, Comedy]","[English, Vietnamese]",[United States of America]
4,Other People's Children,Les Enfants des autres,0,fr,https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",10.518,104,84178,2022-09-21,6.818,190,"[Drama, Comedy]","[French, English]",[France]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,Frantz,Frantz,0,fr,,"In the aftermath of WWI, a young German who gr...",17.411,113,7478354,2016-09-07,7.367,668,"[History, Drama, Romance]","[German, French]","[France, Germany]"
182,The Other Half,The Other Half,0,en,http://theotherhalfmovie.com/,A grief-stricken man and a bipolar woman fall ...,2.871,103,0,2016-12-02,6.300,24,"[Drama, Romance]",[English],[Canada]
183,The Ottoman Lieutenant,The Ottoman Lieutenant,40000000,en,https://www.uphe.com/movies/the-ottoman-lieute...,"Lillie, a determined American woman, ventures ...",16.901,111,413844,2017-03-10,6.169,246,"[Romance, Drama, War]","[Turkish, English]","[Turkey, United States of America]"
184,Art of Love,Romantik Hırsız,0,tr,https://www.netflix.com/title/81689799,After learning that the art thief she has been...,85.682,99,0,2024-03-13,6.495,111,"[Romance, Action, Drama]",[Turkish],[Turkey]


### Merge and Clean the Data for Export

In [26]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(reviews_df, movies_df, on="title")

merged_df



,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"Argentina, 1970s. A desperate young woman goes...",2.890,102,0,2021-10-07,3.000,4,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,"When Jaime, a gay teenager, is sent to live in...",36.659,96,15055,2023-03-24,6.622,37,"[Drama, Romance]","[English, French]","[Canada, United States of America]"
2,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,Emma and Jesse are living the perfect life tog...,11.056,100,37820,2023-04-07,6.500,74,"[Romance, Drama, Comedy]","[English, Spanish]","[Czech Republic, United States of America]"
3,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,...,May Pang lovingly recounts her life in rock & ...,2.466,95,77145,2023-04-13,6.000,2,[Documentary],[English],[United States of America]
4,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,...,Struggling but unapologetically living on her ...,8.610,116,3395595,2023-03-31,6.871,89,"[Drama, Crime]","[English, Spanish, Portuguese]",[United States of America]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,https://www.nytimes.com/2017/05/18/movies/ever...,Amandla Stenberg plays a teenager with an immu...,The New York Times,"subject: Movies;creative_works: Everything, Ev...",2017-05-18T21:02:53+0000,270,"Review: ‘Everything, Everything’ Pits Love Aga...",None,None,"Everything, Everything",...,A teenager who's lived a sheltered life becaus...,22.805,96,61621140,2017-05-18,7.245,3641,"[Drama, Romance]",[English],[United States of America]
131,https://www.nytimes.com/2017/05/11/movies/houn...,This feature debut of the Australian filmmaker...,The New York Times,subject: Movies;creative_works: Hounds of Love...,2017-05-11T20:16:51+0000,280,Review: ‘Hounds of Love’ Is Tense and Deadly D...,None,None,Hounds of Love,...,When Vicki Maloney is randomly abducted from a...,12.241,108,0,2016-10-07,6.100,330,"[Crime, Drama, Horror]",[English],[Australia]
132,https://www.nytimes.com/2017/04/20/movies/the-...,"Oscar Isaac, Christian Bale and Charlotte Le B...",The New York Times,subject: Movies;creative_works: The Promise (M...,2017-04-20T16:49:46+0000,455,Review: ‘The Promise’ Finds a Love Triangle in...,None,None,"A Love Triangle, Near the Front Lines",...,Set during the last days of the Ottoman Empire...,21.376,133,12448676,2016-12-02,6.900,452,"[History, Romance, Drama]","[Armenian, German, English, French]","[Spain, United States of America]"
133,https://www.nytimes.com/2017/03/09/movies/the-...,This moody

In [28]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['genres', 'spoken_languages', 'production_countries']


# Create a list of characters to remove
characters_to_remove = ['[', ']', "'"]


# Loop through the list of columns to fix
for column in columns_to_fix:

    # Convert the column to type 'str'
    merged_df[column] = merged_df[column].astype(str)

    # Loop through characters to remove
    for char in characters_to_remove:
        merged_df[column] = merged_df[column].str.replace(char, '')

# Display the fixed DataFrame
merged_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"Argentina, 1970s. A desperate young woman goes...",2.890,102,0,2021-10-07,3.000,4,"Drama, Mystery, Thriller, Horror",Spanish,Argentina
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,"When Jaime, a gay teenager, is sent to live in...",36.659,96,15055,2023-03-24,6.622,37,"Drama, Romance","English, French","Canada, United States of America"
2,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,Emma and Jesse are living the perfect life tog...,11.056,100,37820,2023-04-07,6.500,74,"Romance, Drama, Comedy","English, Spanish","Czech Republic, United States of America"
3,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,...,May Pang lovingly recounts her life in rock & ...,2.466,95,77145,2023-04-13,6.000,2,Documentary,English,United States of America
4,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,...,Struggling but unapologetically living on her ...,8.610,116,3395595,2023-03-31,6.871,89,"Drama, Crime","English, Spanish, Portuguese",United States of America
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,https://www.nytimes.com/2017/05/18/movies/ever...,Amandla Stenberg plays a teenager with an immu...,The New York Times,"subject: Movies;creative_works: Everything, Ev...",2017-05-18T21:02:53+0000,270,"Review: ‘Everything, Everything’ Pits Love Aga...",None,None,"Everything, Everything",...,A teenager who's lived a sheltered life becaus...,22.805,96,61621140,2017-05-18,7.245,3641,"Drama, Romance",English,United States of America
131,https://www.nytimes.com/2017/05/11/movies/houn...,This feature debut of the Australian filmmaker...,The New York Times,subject: Movies;creative_works: Hounds of Love...,2017-05-11T20:16:51+0000,280,Review: ‘Hounds of Love’ Is Tense and Deadly D...,None,None,Hounds of Love,...,When Vicki Maloney is randomly abducted from a...,12.241,108,0,2016-10-07,6.100,330,"Crime, Drama, Horror",English,Australia
132,https://www.nytimes.com/2017/04/20/movies/the-...,"Oscar Isaac, Christian Bale and Charlotte Le B...",The New York Times,subject: Movies;creative_works: The Promise (M...,2017-04-20T16:49:46+0000,455,Review: ‘The Promise’ Finds a Love Triangle in...,None,None,"A Love Triangle, Near the Front Lines",...,Set during the last days of the Ottoman Empire...,21.376,133,12448676,2016-12-02,6.900,452,"History, Romance, Drama","Armenian, German, English, French","Spain, United States of America"
133,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New 

In [29]:
# Drop "byline.person" column
merged_df.drop("byline.person", axis=1, inplace=True)


In [30]:
# Delete duplicate rows and reset index
merged_df.drop_duplicates(inplace=True)
merged_df.reset_index(drop= True, inplace= True)
merged_df


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"Argentina, 1970s. A desperate young woman goes...",2.890,102,0,2021-10-07,3.000,4,"Drama, Mystery, Thriller, Horror",Spanish,Argentina
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,"When Jaime, a gay teenager, is sent to live in...",36.659,96,15055,2023-03-24,6.622,37,"Drama, Romance","English, French","Canada, United States of America"
2,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,Emma and Jesse are living the perfect life tog...,11.056,100,37820,2023-04-07,6.500,74,"Romance, Drama, Comedy","English, Spanish","Czech Republic, United States of America"
3,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,...,May Pang lovingly recounts her life in rock & ...,2.466,95,77145,2023-04-13,6.000,2,Documentary,English,United States of America
4,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,...,Struggling but unapologetically living on her ...,8.610,116,3395595,2023-03-31,6.871,89,"Drama, Crime","English, Spanish, Portuguese",United States of America
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,https://www.nytimes.com/2017/05/18/movies/ever...,Amandla Stenberg plays a teenager with an immu...,The New York Times,"subject: Movies;creative_works: Everything, Ev...",2017-05-18T21:02:53+0000,270,"Review: ‘Everything, Everything’ Pits Love Aga...",None,None,"Everything, Everything",...,A teenager who's lived a sheltered life becaus...,22.805,96,61621140,2017-05-18,7.245,3641,"Drama, Romance",English,United States of America
124,https://www.nytimes.com/2017/05/11/movies/houn...,This feature debut of the Australian filmmaker...,The New York Times,subject: Movies;creative_works: Hounds of Love...,2017-05-11T20:16:51+0000,280,Review: ‘Hounds of Love’ Is Tense and Deadly D...,None,None,Hounds of Love,...,When Vicki Maloney is randomly abducted from a...,12.241,108,0,2016-10-07,6.100,330,"Crime, Drama, Horror",English,Australia
125,https://www.nytimes.com/2017/04/20/movies/the-...,"Oscar Isaac, Christian Bale and Charlotte Le B...",The New York Times,subject: Movies;creative_works: The Promise (M...,2017-04-20T16:49:46+0000,455,Review: ‘The Promise’ Finds a Love Triangle in...,None,None,"A Love Triangle, Near the Front Lines",...,Set during the last days of the Ottoman Empire...,21.376,133,12448676,2016-12-02,6.900,452,"History, Romance, Drama","Armenian, German, English, French","Spain, United States of America"
126,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New 

In [31]:
merged_df.dtypes

web_url                     object
snippet                     object
source                      object
keywords                    object
pub_date                    object
word_count                   int64
headline.main               object
headline.kicker             object
headline.content_kicker     object
headline.print_headline     object
headline.name               object
headline.seo                object
headline.sub                object
byline.original             object
byline.organization         object
title                       object
original_title              object
budget                       int64
original_language           object
homepage                    object
overview                    object
popularity                 float64
runtime                      int64
revenue                      int64
release_date                object
vote_average               float64
vote_count                   int64
genres                      object
spoken_languages    

In [32]:
# Export data to CSV without the index
merged_df.to_csv('merged_data.csv', index=False)